In [2]:
import pandas as pd
import numpy as np
from pyAudioAnalysis import audioBasicIO #A
from pyAudioAnalysis import audioFeatureExtraction #B
import matplotlib.pyplot as plt
import os
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics.cluster import normalized_mutual_info_score

C:\ProgramData\Anaconda3\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [4]:
features= pd.read_csv('features.txt') 
features.head()

,Unnamed: 0,G#,G,F#,F,E,D#,D,C#,C,B,A#,A,filename,group
0,0,0.004988,0.000000,0.354115,0.007481,0.064838,0.000000,0.000000,0.271820,0.000000,0.000000,0.029925,0.266833,50_Minutes-Colours.wav,alternative
1,1,0.002494,0.286783,0.000000,0.000000,0.019950,0.022444,0.299252,0.000000,0.000000,0.000000,0.069825,0.299252,50_Minutes-Nightwalk.wav,alternative
2,2,0.037406,0.000000,0.047382,0.032419,0.346633,0.000000,0.000000,0.241895,0.000000,0.000000,0.214464,0.079800,50_Minutes-The_Great_Wall.wav,alternative
3,3,0.184539,0.009975,0.137157,0.089776,0.092269,0.027431,0.054863,0.012469,0.027431,0.054863,0.286783,0.022444,Abigail_Lapell-Dress_Rehearsal.wav,alternative
4,4,0.154613,0.054863,0.336658,0.014963,0.054863,0.194514,0.064838,0.004988,0.017456,0.069825,0.027431,0.004988,Albino_Catfish-Ice_Cream.wav,alternative


In [7]:
features=features.drop(features[features.group=='alternative'].index)
features=features.drop(features[features.group=='blues'].index)
features=features.drop(features[features.group=='electronic'].index)
# features=features.drop(features[features.group=='jazz'].index)
# features=features.drop(features[features.group=='funksoulrnb'].index)
features.head()

,Unnamed: 0,G#,G,F#,F,E,D#,D,C#,C,B,A#,A,filename,group
378,0,0.039900,0.000000,0.441397,0.037406,0.087282,0.007481,0.264339,0.000000,0.000000,0.000000,0.117207,0.004988,3_Blind_Mice-Emily_Has_Compassion_Fatigue.wav,folkcountry
379,1,0.039900,0.000000,0.236908,0.012469,0.132170,0.019950,0.204489,0.002494,0.002494,0.017456,0.266833,0.064838,6_gun_5-Gone_Too_Far.wav,folkcountry
380,2,0.022444,0.027431,0.264339,0.104738,0.169576,0.039900,0.179551,0.014963,0.002494,0.057357,0.114713,0.002494,Aaron_Cheney-If_He_Were_Alive_Today__He_d_Prob...,folkcountry
381,3,0.039900,0.079800,0.281796,0.064838,0.084788,0.069825,0.127182,0.022444,0.007481,0.089776,0.069825,0.062344,Aaron_Cheney-Intimate_Strangers.wav,folkcountry
382,4,0.029925,0.007481,0.169576,0.034913,0.097257,0.000000,0.022444,0.359102,0.002494,0.032419,0.072319,0.172070,Aaron_Wilkinson-Osho_mask.wav,folkcountry


In [8]:
x= features.iloc[:,1:-2]
y=features.iloc[:,-1]
x.head()

,G#,G,F#,F,E,D#,D,C#,C,B,A#,A
378,0.039900,0.000000,0.441397,0.037406,0.087282,0.007481,0.264339,0.000000,0.000000,0.000000,0.117207,0.004988
379,0.039900,0.000000,0.236908,0.012469,0.132170,0.019950,0.204489,0.002494,0.002494,0.017456,0.266833,0.064838
380,0.022444,0.027431,0.264339,0.104738,0.169576,0.039900,0.179551,0.014963,0.002494,0.057357,0.114713,0.002494
381,0.039900,0.079800,0.281796,0.064838,0.084788,0.069825,0.127182,0.022444,0.007481,0.089776,0.069825,0.062344
382,0.029925,0.007481,0.169576,0.034913,0.097257,0.000000,0.022444,0.359102,0.002494,0.032419,0.072319,0.172070


In [9]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
y=encoder.fit_transform(y)

In [10]:
np.unique(y)

array([0, 1, 2, 3, 4, 5])

In [11]:
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(n_clusters=6,init='k-means++', random_state=0).fit(x)
# kmeans.labels_

In [12]:
test=pd.DataFrame()
test['y']=y
test['ny']=kmeans.labels_
test.head()
groupedtest=pd.DataFrame(test.groupby(by=['y','ny']).size().reset_index(name='count'))

In [13]:
print(adjusted_rand_score(test.y,test.ny))

0.020861400469275725


In [14]:
test.head()

,y,ny
0,0,0
1,0,1
2,0,1
3,0,0
4,0,4


In [15]:
import itertools 
perm = itertools.permutations(np.unique(y)) 
_totalperm=[]
  
for i in list(perm): 
    _totalperm.append(list(i)) 

In [16]:
from sklearn import metrics as m
accuracy=np.zeros((len(_totalperm)))
values=[]

for i in range(len(_totalperm)): 
    print(i)
    newcolvals=np.zeros((test.shape[0]))
    for j in range(test.shape[0]):
        newcolvals[j]=(_totalperm[i])[test.ny[j]] 
        
    accval=m.accuracy_score(test.y,newcolvals)
    accuracy[i]=(accval)
    values.append(list(_totalperm[i]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [17]:
np.max(accuracy)

0.27519893899204245